In [12]:
; Add code libraries

(require '[clojupyter.misc.helper :as helper])

(helper/add-dependencies '[org.clojure/data.csv "1.0.0"])
(helper/add-dependencies '[org.clojure/data.json "1.0.0"])
(helper/add-dependencies '[clj-http/clj-http "3.10.1"])
(helper/add-dependencies '[org.apache.commons/commons-math3 "3.6.1"])

(require '[clojure.string :as str]
         '[clojure.set :as set]
         '[clojure.pprint :as pp]
         '[clojure.java.io :as io]
         '[clojure.data.csv :as csv]
         '[clojure.data.json :as json]
         '[clj-http.client :as http])
         
(import 'java.io.FileWriter
        'java.net.URLEncoder
        'java.math.RoundingMode
        'org.apache.commons.math3.stat.regression.SimpleRegression)

org.apache.commons.math3.stat.regression.SimpleRegression

In [2]:
; Define convenience functions

; Convert the CSV structure to a list-of-maps structure.
(defn to-maps [csv-data]
    (map zipmap (->> (first csv-data)
                    (map keyword)
                    repeat)
                (rest csv-data)))

; Map the name of a SPARQL service to its URL.
(def service-urls {:scotgov "http://statistics.gov.scot/sparql"
                   :wikidata "https://query.wikidata.org/sparql"})

; Ask statistic.gov.scot to execute the given SPARQL query
; and return its result as a list-of-maps.
(defn exec-query [service-name sparql]
    (->> (http/post (service-name service-urls) 
                    {:body (str "query=" (URLEncoder/encode sparql)) 
                    :headers {"Accept" "text/csv" 
                              "Content-Type" "application/x-www-form-urlencoded"} 
                    :debug false})
        :body
        csv/read-csv
        to-maps))
        
; Compute 'the trend of y'.
; (Returns the gradient of a linear approximation to the curve decribed by xy-pairs.)
(defn trend [xy-pairs]
    (let [regression (SimpleRegression. true)]
        (doseq [[x y] xy-pairs]
            (.addData regression x y))
        (.getSlope regression)))

#'user/trend

In [3]:
; Query for the household waste solids data

(def sparql "

PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pdmx: <http://purl.org/linked-data/sdmx/2009/dimension#>
PREFIX sdmx: <http://statistics.gov.scot/def/dimension/>
PREFIX snum: <http://statistics.gov.scot/def/measure-properties/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT 
    ?year
    ?area
    ?endState
    ?material
    ?tonnes 
    
WHERE {
  
    ?tonnageObs qb:dataSet <http://statistics.gov.scot/data/household-waste> .
    ?tonnageObs pdmx:refArea ?areaUri .
    ?tonnageObs pdmx:refPeriod ?periodUri .
    ?tonnageObs sdmx:wasteCategory ?wasteCategoryUri .
    ?tonnageObs sdmx:wasteManagement ?wasteManagementUri .
    ?tonnageObs snum:count ?tonnes .
  
    ?areaUri rdfs:label ?area .
    ?periodUri rdfs:label ?year .
    ?wasteCategoryUri rdfs:label ?material .
    ?wasteManagementUri rdfs:label ?endState .
}
")

(def household-waste-solids 
    (->> sparql
        (exec-query :scotgov)
        (map #(assoc % :year (.intValue (bigdec (:year %)))
                       :tonnes (.doubleValue (bigdec (:tonnes %)))))))

(println (count household-waste-solids ) "rows")

36432 rows


nil

In [4]:
; Print a sample

(def ks [:year :area :endState :material :tonnes])
(pp/print-table ks (repeatedly 10 #(rand-nth household-waste-solids )))


| :year |                 :area |                         :endState |                                                                             :material | :tonnes |
|-------+-----------------------+-----------------------------------+---------------------------------------------------------------------------------------+---------|
|  2015 |         East Ayrshire | Other Diversion (pre 2014 method) |                                                           Animal and mixed food waste |     0.0 |
|  2017 |     Perth and Kinross |                        Landfilled |                                                                          Glass wastes |     0.0 |
|  2018 |            Inverclyde | Other Diversion (pre 2014 method) |                                                              Metallic wastes, ferrous |     0.0 |
|  2013 |         East Ayrshire |        Recycled (pre 2014 method) |                                                     Health care and biological wastes |  

nil

In [5]:
; Query for the population data

(def sparql "

PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pdmx: <http://purl.org/linked-data/sdmx/2009/dimension#>
PREFIX sdmx: <http://statistics.gov.scot/def/dimension/>
PREFIX snum: <http://statistics.gov.scot/def/measure-properties/>
PREFIX uent: <http://statistics.data.gov.uk/def/statistical-entity#>
PREFIX ugeo: <http://statistics.data.gov.uk/def/statistical-geography#>

SELECT 
  ?year
  ?area
  ?population

WHERE {
  VALUES ?areaType { 
           <http://statistics.gov.scot/id/statistical-entity/S92>
           <http://statistics.gov.scot/id/statistical-entity/S12> }
           
  ?areaUri uent:code ?areaType;
           ugeo:status 'Live' ;
           rdfs:label ?area .
           
  ?populationUri qb:dataSet <http://statistics.gov.scot/data/population-estimates-current-geographic-boundaries> ;
                 pdmx:refArea ?areaUri ;
                 pdmx:refPeriod ?periodUri ;
                 sdmx:age <http://statistics.gov.scot/def/concept/age/all> ;
                 sdmx:sex <http://statistics.gov.scot/def/concept/sex/all> ;
                 snum:count ?population .
  
  ?periodUri rdfs:label ?year .
}
")

(def population
    (->> sparql
        (exec-query :scotgov)
        (map #(assoc % :year (.intValue (bigdec (:year %)))
                       :population (.intValue (bigdec (:population %)))))))

(println (count population ) "rows")

627 rows


nil

In [6]:
; Print a sample

(def ks [:year :area :population])
(pp/print-table ks (repeatedly 10 #(rand-nth population )))


| :year |               :area | :population |
|-------+---------------------+-------------|
|  2015 |       Aberdeenshire |      261960 |
|  2005 |       Aberdeenshire |      237570 |
|  2007 | West Dunbartonshire |       91370 |
|  2017 |             Falkirk |      160130 |
|  2013 |        Glasgow City |      596520 |
|  2015 |   East Renfrewshire |       92940 |
|  2003 |            Scotland |     5068500 |
|  2005 |               Moray |       90100 |
|  2007 |               Moray |       91440 |
|  2017 |                Fife |      371410 |


nil

In [8]:
; Query for the household data

(def sparql "

PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pdmx: <http://purl.org/linked-data/sdmx/2009/dimension#>
PREFIX sdmx: <http://statistics.gov.scot/def/dimension/>
PREFIX snum: <http://statistics.gov.scot/def/measure-properties/>
PREFIX uent: <http://statistics.data.gov.uk/def/statistical-entity#>
PREFIX ugeo: <http://statistics.data.gov.uk/def/statistical-geography#>

SELECT 
  ?year
  ?area
  ?count

WHERE {
  VALUES ?areaType { 
           <http://statistics.gov.scot/id/statistical-entity/S92>
           <http://statistics.gov.scot/id/statistical-entity/S12> }
           
  ?areaUri uent:code ?areaType;
           ugeo:status 'Live' ;
           rdfs:label ?area .
           
  ?householdUri qb:dataSet <http://statistics.gov.scot/data/mid-year-household-estimates> ;
                 pdmx:refArea ?areaUri ;
                 pdmx:refPeriod ?periodUri ;
                 snum:count ?count .
  
  ?periodUri rdfs:label ?year .
}
")

(def household
    (->> sparql
        (exec-query :scotgov)
        (map #(assoc % :year (.intValue (bigdec (:year %)))
                       :count (.intValue (bigdec (:count %)))))))

(println (count household ) "rows")

627 rows


nil

In [9]:
; Print a sample

(def ks [:year :area :count])
(pp/print-table ks (repeatedly 10 #(rand-nth household )))


| :year |             :area | :count |
|-------+-------------------+--------|
|  2013 |             Angus |  52413 |
|  2007 |     East Ayrshire |  52425 |
|  2001 |       Dundee City |  66854 |
|  2017 |        Inverclyde |  37651 |
|  2005 |      Renfrewshire |  77483 |
|  2002 |   Argyll and Bute |  39135 |
|  2012 | North Lanarkshire | 146905 |
|  2008 |       Dundee City |  68594 |
|  2017 |    South Ayrshire |  52104 |
|  2008 |          Stirling |  37120 |


nil

In [19]:
; Query for the CO2e data

(def sparql "

SELECT 
    ?year 
    ?councilAreaLabel 
    ?CO2eAmount
    
WHERE {
  ?councilArea wdt:P31 wd:Q15060255 . # Scottish council area
    
  BIND(strafter(str(?councilArea), 'http://www.wikidata.org/entity/') as ?qid)
    
  ?councilArea p:P5991 ?CO2e . 
  ?CO2e psv:P5991 ?CO2eQuantity ;
        pq:P585 ?date ;
        pq:P828 wd:Q180388 ; # 'has cause' 'waste management'
        pq:P828 wd:Q259059 . # 'has cause' 'household' .
  ?CO2eQuantity wikibase:quantityAmount ?CO2eAmount.
    
  BIND(YEAR(?date) as ?year)
    
  SERVICE wikibase:label { bd:serviceParam wikibase:language '[AUTO_LANGUAGE],en' . }
}
")

(def co2e
    (->> sparql
        (exec-query :wikidata)
        (map #(set/rename-keys % {:councilAreaLabel :area
                                  :CO2eAmount :co2e}))
        (map #(assoc % :year (.intValue (bigdec (:year %)))
                       :co2e (.doubleValue (bigdec (:co2e %)))))
        (map #(let [area (:area %)] ;; adjust Wikidata values to be in keeping with those from Scottish gov
                  (cond 
                    (= area "Outer Hebrides") (assoc % :area "Na h-Eileanan Siar")
                    :else %)))
        
        ))

(println (count co2e ) "rows")

64 rows


nil

In [20]:
; Print a sample

(def ks [:year :area :co2e])
(pp/print-table ks (repeatedly 10 #(rand-nth co2e)))


| :year |                 :area |     :co2e |
|-------+-----------------------+-----------|
|  2017 |                 Angus | 115899.85 |
|  2017 |         Aberdeenshire | 325282.92 |
|  2017 |   East Dunbartonshire | 130754.58 |
|  2017 |         Aberdeen City | 198579.67 |
|  2017 |           Dundee City | 154653.39 |
|  2017 |                 Angus | 115899.85 |
|  2018 | Dumfries and Galloway | 227001.47 |
|  2017 |   East Dunbartonshire | 130754.58 |
|  2017 |        Orkney Islands |  29072.83 |
|  2018 |      Shetland Islands |  31049.37 |


nil

({:area "Scotland", :year 2017, :co2e 5863248.86} {:area "Scotland", :year 2018, :co2e 5759986.49})

In [27]:
; Assemble the target data structure

(def population-indexed 
    (group-by (juxt :year :area) population))
    
(def household-indexed 
    (group-by (juxt :year :area) household))


    

(def target-data-struct
    (concat
        (->> household-waste-solids
            (filter #(not (or ;; could be useful, so leave in but hide on the page
                              ;; (= "Scotland" (:area %))
                              ;; duplicated by & could clash with summing on the page, so remove
                              (= "Waste Generated" (:endState %))
                              (= "Total Waste" (:material %))
                              ;; confusingly duplicated in the non pre 2014 data, so remove
                              (= "Other Diversion (pre 2014 method)" (:endState %)) ;; remove
                              (= "Recycled (pre 2014 method)" (:endState %)))))
            (map #(if-let [population (-> population-indexed (get [(:year %) (:area %)]) first :population)]
                    (assoc % :tonnesPerCitizen (/ (:tonnes %) population))
                    %))
            (map #(if-let [count (-> household-indexed (get [(:year %) (:area %)]) first :count)]
                    (assoc % :tonnesPerHousehold (/ (:tonnes %) count))
                    %)))
         (->> co2e
            (concat ;; the entries for Scotland as a whole
                (let [per-year (group-by :year co2e)]
                    (for [year (keys per-year)]
                        {:area "Scotland"
                         :year year
                         :co2e (apply + (map :co2e (get per-year year)))})))
            (map #(if-let [population (-> population-indexed (get [(:year %) (:area %)]) first :population)]
                    (assoc % :co2ePerCitizen (/ (:co2e %) population))
                    %))
            (map #(if-let [count (-> household-indexed (get [(:year %) (:area %)]) first :count)]
                    (assoc % :co2ePerHousehold (/ (:co2e %) count))
                    %)))))
         
(println (count target-data-struct) "rows")

17490 rows


nil

In [28]:
; Write, as JSON, to a file
         
(def file (io/file "dx-data.json"))
(binding [*out* (FileWriter. file)]
    (json/pprint target-data-struct))
(println "wrote the JSON file" (.getAbsolutePath file))

wrote the JSON file /Users/amc/workspace/data-commons-scotland/dcs-shorts/pivot-drilldown-and-plot/dx-data.json


nil